# Pytorch CNN

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 1

d:\anaconda3\envs\pyTorchGPU\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

In [3]:
# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transforms.ToTensor(), download=True)

In [4]:
# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [5]:
class Model(nn.Module):
# TODO:define model
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Sequential(torch.nn.Conv2d(1, 64, 3, 1, 1),#输入1通道，输出64通道，使用了64个3*3的卷积核，步长为1，填充数为1
                                         torch.nn.ReLU(), 
                                         torch.nn.Conv2d(64, 128, 3, 1, 1),#(输入 - kernel + 1 + 2*填充)/步长 = (28 - 3 + 1 + 1 * 2) / 1
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(2,2)) # 28 * 28 -> 14 * 14
        self.dense = torch.nn.Sequential(torch.nn.Linear(128 * 14 * 14, 1024),
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p = 0.5), # 防止过拟合
                                         torch.nn.Linear(1024, 10)) # 输出为分类数量 Linear(input, output)上方同理
        
    def forward(self, x):
        x = self.conv1(x)
        x = x.view(-1, 14 * 14 * 128)
        x = self.dense(x)
        return x


In [6]:
model = Model()
print(model)



Model(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [7]:
# TODO:define loss function and optimiter

device = torch.device('cuda')
model = Model().to(device)

cost = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters())

# model.load_state_dict(torch.load('model_parameter.pkl'))

In [8]:


# train and evaluate
for epoch in range(NUM_EPOCHS):
    sum_loss = 0.0
    train_correct = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        X_train = images
        y_train = labels
        X_train, y_train = torch.autograd.Variable(X_train).cuda(), torch.autograd.Variable(y_train).cuda()
        optimizer.zero_grad()
        outputs = model(X_train.cuda())


        loss = cost(outputs, labels.cuda())
        loss.backward()

        _, pred = torch.max(outputs.data, 1)
        optimizer.step()
        
        sum_loss += loss.data
        train_correct += torch.sum(pred == y_train.data)
    print('[%d,%d] loss:%.03f' % (epoch + 1, NUM_EPOCHS, sum_loss / len(train_loader)))
    print('correct:%.03f%%' % (100 * train_correct / len(train_dataset)))
torch.save(model.state_dict(), "model_parameter.pkl")        

  

100%|██████████| 468/468 [01:01<00:00,  7.56it/s]


[1,1] loss:0.155
correct:94.950%


In [9]:
test_correct = 0
for images, labels in tqdm(test_loader): 
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
    X_test = images
    y_test = labels
    X_test, y_test = torch.autograd.Variable(X_test).cuda(), torch.autograd.Variable(y_test).cuda()
    outputs = model(X_test.cuda())
    _, pred = torch.max(outputs.data, 1)
    test_correct += torch.sum(pred == y_test.data)
 
print("correct:%.3f%%" % (100 * test_correct / len(test_dataset)))
    

100%|██████████| 78/78 [00:02<00:00, 26.43it/s]

correct:97.980%


In [ ]:
print(len(test_dataset))
print(test_correct)
print("correct:%.3f%%" % (100 * test_correct / len(test_dataset)))

In [ ]:
data_loader_test = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size = 4,
                                          shuffle = True)
X_test, y_test = next(iter(data_loader_test))
inputs = torch.autograd.Variable(X_test)
pred = model(inputs)
_, pred = torch.max(pred, 1)

print("pred:",[i for i in pred.data])
print("Ture:",[i for i in y_test])

img = torchvision.utils.make_grid(X_test)
img = img.numpy().transpose(1,2,0)
std = [0.5,0.5,0.5]
mean = [0.5,0.5,0.5]
img = img*std+mean

import matplotlib.pyplot as plt

plt.imshow(img)